# Alpha Model

## Inputs & Installs

In [ ]:
!pip install wandb

In [ ]:
!wandb login aa0b9ecff47af231f410704977e504d7928ffb05

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
import wandb

wandb.init(project="danish-inflection")

wandb: Currently logged in as: ignacioct_ (ignacio_at_ai). Use `wandb login --relogin` to force relogin


In [1]:
!pip install fairseq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.3/106.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.3/288.3 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for fairseq: filename=fairseq-0.12.2-cp310-cp310-linux_x86_64.whl size=11291796 sha256=a461f9e357cb46134ce5bd78201afacb50c464928c4dc5d9d1521e021075cb8b
  Stored in directory: /root/.cache/pip/wheels/e4/35/55/9c66f65ec7c83fd6fbc2b9502a0ac81b2448a1196159dacc32
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.wh

In [2]:
!pip install tensorboardX

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.2 MB/s eta 0:00:00


## Preprocess the data

In [13]:
!bash ./preprocess.sh dan

2024-01-02 21:03:55.754547: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 21:03:55.754644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 21:03:55.756908: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 21:03:55.768246: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 21:03:57.638333: W tensorflow/compiler/tf2

## Train

Train with default parameters, roughly the baseline in SIGMORPHON 2020 shared task
Let this run until the loss on the validation (dev) test no longer improves. (Maybe 10 minutes with a GPU).

In [ ]:
!bash ./train.sh dan --patience 1

2024-01-01 19:37:36.672152: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 19:37:36.672235: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 19:37:36.674361: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 19:37:36.694209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 19:37:39.643565: W tensorflow/compiler/tf2

## Generate predictions on test data

Generate predictions on test data - read in all the inputs from tst.esp.input and generate outputs to the file tst.esp.output (this is slow and takes about a minute)

In [ ]:
!fairseq-interactive data-bin/dan/ --source-lang=dan.input --target-lang=dan.output --path=checkpoints/dan-models/checkpoint_best.pt --input=tst.dan.input | grep -P "D-[0-9]+" | cut -f3 > tst.dan.output

2024-01-01 21:40:20.491792: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-01 21:40:20.491900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-01 21:40:20.499200: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-01 21:40:20.519614: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-01 21:40:22.910911: W tensorflow/compiler/tf2

In [3]:
# Read in the generated outputs and inputs and display the first 20 side-by-side
linesinput = [l.strip() for l in open("tst.dan.input")]
linesoutput = [l.strip() for l in open("tst.dan.output")]
tuple(zip(linesinput, linesoutput))[:20] # Look at 20 first test inputs and predicted outputs

(('< i n d b r u d > N DEF NOM PL', '< i n d b r u d d e n e s >'),
 ('< c i g a r e t s k o d > N DEF NOM PL',
  '< c i g a r e t s k o d d e n e s >'),
 ('< r o t t e > N INDF GEN PL', '< r o t t e r s >'),
 ('< m a s s ø r > N INDF NOM SG', '< m a s s ø r >'),
 ('< n u l > N INDF NOM PL', '< n u l l e r >'),
 ('< d a t a > N INDF GEN PL', '< d a t a s >'),
 ('< k r æ s e > V PASS IND PST', '< k r æ s e d e s >'),
 ('< b r æ k j e r n > N DEF NOM SG', '< b r æ k j e r n e t s >'),
 ('< w c - r u l l e > N DEF NOM SG', '< w c - r u l l e n s >'),
 ('< t e p o s e > N DEF NOM SG', '< t e p o s e n s >'),
 ('< p a t i e n t > N DEF NOM SG', '< p a t i e n t e n s >'),
 ('< s t ø r r e l s e > N DEF NOM PL', '< s t ø r r e l s e r n e s >'),
 ('< b y g n i n g > N INDF GEN SG', '< b y g n i n g s >'),
 ('< k l a v i a t u r > N INDF NOM PL', '< k l a v i a t u r e r >'),
 ('< d ø g n > N DEF NOM PL', '< d ø g n e n e s >'),
 ('< p a r k e r i n g s g a r a g e > N INDF GEN SG',
  '< p a 

## Calculate test accuracy and Levensthein distance

In [14]:
# Creating the predictions from the checkpoint
!fairseq-interactive data-bin/dan/ --source-lang=dan.input --target-lang=dan.output --path=checkpoint_best.pt --input=tst.dan.input | grep -P "D-[0-9]+" | cut -f3 > tst.dan.prediction

2024-01-02 21:04:23.114571: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-02 21:04:23.114656: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-02 21:04:23.116171: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-02 21:04:23.128370: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-02 21:04:24.824427: W tensorflow/compiler/tf2

In [4]:
# Calculating the accuracy between the ground truth and the predictions
linesprediction = [l.strip() for l in open("tst.dan.prediction")]
linesground = [l.strip() for l in open("tst.dan.output")]

# Checking if both files have the same number of lines
assert sum(1 for _ in enumerate(linesprediction)) == sum(1 for _ in enumerate(linesground))
assert sum(1 for _ in enumerate(linesprediction)) != 0
assert sum(1 for _ in enumerate(linesground)) != 0

hits = 0
lines = 0


for pred, ground in zip(linesprediction, linesground):
  if pred == ground:
    hits += 1
  lines += 1

print("Accuracy: " + str(hits/lines))


Accuracy: 0.8292682926829268


In [7]:
# Calculating the Levenshtein distance between the ground truth and the predictions

# Function from ChatGPT
def levenshtein_distance(str1, str2):
    len_str1 = len(str1) + 1
    len_str2 = len(str2) + 1

    # Create a matrix to store the distances
    matrix = [[0 for _ in range(len_str2)] for _ in range(len_str1)]

    # Initialize the matrix
    for i in range(len_str1):
        matrix[i][0] = i
    for j in range(len_str2):
        matrix[0][j] = j

    # Fill in the matrix
    for i in range(1, len_str1):
        for j in range(1, len_str2):
            cost = 0 if str1[i - 1] == str2[j - 1] else 1
            matrix[i][j] = min(
                matrix[i - 1][j] + 1,        # Deletion
                matrix[i][j - 1] + 1,        # Insertion
                matrix[i - 1][j - 1] + cost  # Substitution
            )

    # The bottom-right cell contains the Levenshtein distance
    return matrix[-1][-1]

linesprediction = [l.strip() for l in open("tst.dan.prediction")]
linesground = [l.strip() for l in open("tst.dan.output")]

# Checking if both files have the same number of lines
assert sum(1 for _ in enumerate(linesprediction)) == sum(1 for _ in enumerate(linesground))
assert sum(1 for _ in enumerate(linesprediction)) != 0
assert sum(1 for _ in enumerate(linesground)) != 0

distances = 0
lines = 0


for pred, ground in zip(linesprediction, linesground):

  distances += levenshtein_distance(pred, ground)
  lines += 1

print("Levenshtein distance: " + str(distances/lines))

2
2
0
0
6
4
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
2
0
0
1
0
0
0
2
0
0
0
0
0
0
2
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
2
2
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
4
0
1
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
1
0
0
4
0
0
0
0
4
0
0
3
0
0
0
0
0
1
0
3
2
0
0
0
0
0
0
4
0
0
2
4
0
0
0
0
0
0
0
0
0
2
5
0
0
0
0
0
4
0
0
0
0
1
4
0
0
1
0
0
4
0
0
4
0
0
0
5
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
0
4
2
0
0
1
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
2
0
2
0
0
0
0
0
2
0
0
0
0
0
0
2
0
0
0
2
1
0
4
2
0
0
0
2
0
0
7
0
0
0
0
0
0
0
0
0
0
0
5
0
0
4
0
0
0
0
0
0
0
0
0
0
1
2
0
0
0
0
0
0
0
0
2
12
0
0
0
0
0
0
0
4
0
0
0
2
0
0
0
6
0
4
0
0
2
0
0
0
0
0
0
0
4
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
4
0
0
0
0
0
0
2
0
0
0
0
0
2
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
18
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
4
6
0
0
0
0
1
0
0
0
1
0
0
4
0
0
0
0
2
0
0
0
0
0
2
1
0
1
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
5
0
2
0
0
0
0
0
0
0
0
0
0
0
0
3
0
0
0
0


Distance = 0: The strings are identical. No edits are needed.

Distance = 1: The strings are very similar. Typically, this means either a single insertion, deletion, or substitution is required to make them identical. For example, "cat" and "cot" have a Levenshtein distance of 1 because you can transform one into the other by changing a single character.

Distance > 1: As the distance increases, the dissimilarity between the strings also increases. A distance of 2 or more indicates a greater degree of dissimilarity, involving multiple edits.